In [2]:
# %%
# Install required libraries (run this cell only once)
!pip install pandas numpy folium ipython

# %%
# Imports
import pandas as pd
import numpy as np
import folium
from folium.plugins import MarkerCluster
from IPython.display import IFrame

# %%
# Load datasets (ensure data1.csv and data2.csv are in the same folder)
data1 = pd.read_csv('data1.csv')
data2 = pd.read_csv('data2.csv')

# %%
# Data Cleaning & Preprocessing
# 1. Clean 'rate' column, handling '-' and 'NEW'
data1['rate'] = data1['rate'].replace({'-': pd.NA, 'NEW': pd.NA})
data1['rate'] = data1['rate'].str.replace('/5', '', regex=False)
data1['rate'] = pd.to_numeric(data1['rate'], errors='coerce')
data1['rate'] = data1['rate'].fillna(data1['rate'].median())

# 2. Clean 'approx_costfor_two_people'
data1['approx_costfor_two_people'] = data1['approx_costfor_two_people'].astype(str).str.replace(',', '')
data1['approx_costfor_two_people'] = pd.to_numeric(data1['approx_costfor_two_people'], errors='coerce')
data1['approx_costfor_two_people'] = data1['approx_costfor_two_people'].fillna(data1['approx_costfor_two_people'].median())

# 3. Fill missing text values
data1['dish_liked'].fillna('Not Available', inplace=True)
data1['cuisines'].fillna('Other', inplace=True)
data1['rest_type'].fillna('Unknown', inplace=True)

# 4. Fill and convert 'votes'
data1['votes'] = data1['votes'].fillna(data1['votes'].median()).astype(int)

# 5. Encode binary columns
data1['online_order'] = data1['online_order'].map({'Yes': 1, 'No': 0})
data1['book_table'] = data1['book_table'].map({'Yes': 1, 'No': 0})

# %%
# Merge with location data
merged_df = pd.merge(data1, data2, on='listed_incity', how='left')

# Confirm merge
print("Merged DataFrame shape:", merged_df.shape)

# %%
# 1) Restaurant Density Map - create and save as HTML
blr_map = folium.Map(location=[12.9716, 77.5946], zoom_start=12)
marker_cluster = MarkerCluster().add_to(blr_map)
for _, row in merged_df.iterrows():
    lat, lon = row['Latitude'], row['Longitude']
    if pd.notnull(lat) and pd.notnull(lon):
        folium.Marker(
            location=[lat, lon],
            popup=f"{row['listed_incity']} | Rate: {row['rate']} | ₹{row['approx_costfor_two_people']}",
            tooltip=row['listed_incity']
        ).add_to(marker_cluster)
# Save and display
blr_map.save('restaurant_density.html')
IFrame('restaurant_density.html', width=900, height=600)

# %%
# 2) Italian Cuisine Map - filter, create, and save as HTML
italian_df = merged_df[merged_df['cuisines'].str.contains('Italian', case=False, na=False)]
italian_map = folium.Map(location=[12.9716, 77.5946], zoom_start=12)
for _, row in italian_df.iterrows():
    lat, lon = row['Latitude'], row['Longitude']
    if pd.notnull(lat) and pd.notnull(lon):
        folium.Marker(
            location=[lat, lon],
            popup=f"{row['listed_incity']} | {row['cuisines']} | Rate: {row['rate']}",
            tooltip=row['listed_incity'],
            icon=folium.Icon(color='purple')
        ).add_to(italian_map)
# Save and display

italian_map.save('italian_restaurants.html')
IFrame('italian_restaurants.html', width=900, height=600)


Defaulting to user installation because normal site-packages is not writeable
Merged DataFrame shape: (51717, 12)
